In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import matplotlib.pylab as pylab
import cartopy.feature as cfeature
from endgame import hist2d
import scipy.stats as stats
import xarray as xr
import ipdb
import glob
import itertools
import numpy.ma as ma
from utils import u_statistics as u_stat
from scipy.stats import gaussian_kde
import pickle as pkl
from utils import u_plot as uplot, u_darrays as uda
import pandas as pd
import os
import xarray as xr

from utils import constants as cnst

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [1]:
cd ..


/home/ck/pythonWorkspace/proj_CEH


Loads bulk statistics derived from having cut out storms (saved in MCSfiles), OBS includes rainfall from TRMM radar

In [3]:
cmip_path = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/'

In [4]:
def get_model_name(path):
    names = []
    files = glob.glob(path+'/*.nc')
    for f in files:
        fname = os.path.basename(f)
        #ipdb.set_trace()
        model = fname.split('_')[2]
        names.append(model)
    return names

In [5]:
shear_names = get_model_name('/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100')

In [6]:
tcw_names = get_model_name('/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100')

In [7]:
model_names = list(set(shear_names).intersection(set(tcw_names)))

In [8]:
len(model_names)

38

In [9]:
model_names

['BNU-ESM',
 'FIO-ESM',
 'CanESM2',
 'IPSL-CM5B-LR',
 'CESM1-BGC',
 'NorESM1-M',
 'HadGEM2-ES',
 'CSIRO-Mk3-6-0',
 'CESM1-CAM5',
 'GFDL-ESM2M',
 'CMCC-CMS',
 'GISS-E2-H',
 'MIROC5',
 'CNRM-CM5',
 'CCSM4',
 'bcc-csm1-1',
 'GFDL-ESM2G',
 'MIROC-ESM',
 'inmcm4',
 'IPSL-CM5A-MR',
 'MRI-CGCM3',
 'ACCESS1-3',
 'CMCC-CESM',
 'CMCC-CM',
 'HadGEM2-CC',
 'IPSL-CM5A-LR',
 'FGOALS-g2',
 'NorESM1-ME',
 'ACCESS1-0',
 'GISS-E2-R-CC',
 'MPI-ESM-LR',
 'MIROC-ESM-CHEM',
 'bcc-csm1-1-m',
 'MPI-ESM-MR',
 'GISS-E2-R',
 'GFDL-CM3',
 'GISS-E2-H-CC',
 'MRI-ESM1']

In [10]:
base = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/'
shear_path_hist = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/'
shear_path_fut = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/'
tcw_path_hist = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/'
tcw_path_fut = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/'

lpaths = [('shear', shear_path_hist, shear_path_fut, 'unknown'), ('tcw', tcw_path_hist, tcw_path_fut, 'prw')]

sahel = [-10,15,9,19]
x=2
y=1.1
ouaga=[-1.52-x,-1.52+x,12.38-y,12.38+y]  #12.3714° N, 1.5197° W
niamey = [2.13-x,2.13+x, 13.51-y,13.51+y] #13.5116° N, 2.1254° E
timbuktu = [ -3-x,-3+x, 16.77-y,16.77+y] #16.7666° N, 3.0026° W
bamako = [-8-x, -8+x, 12.6-y, 12.6+y]
dakar = [-16.9-x, -16.9+x, 14.8-y, 14.8+y]

dnames = [('dakar', dakar)] #('bamako', bamako)] #('sahel', sahel), ('ouaga',  ouaga), ('niamey', niamey), ('timbuktu',  timbuktu) ]
for regions in dnames:
    dic30 = {'model_name' : [], 'shear_mean' : [], 'tcw_mean' : [], 'tcw_hist' : [], 'shear_hist' : []}
    dic40 = {'model_name' : [], 'shear_mean' : [], 'tcw_mean' : [], 'tcw_hist' : [], 'shear_hist' : []}
    dic60 = {'model_name' : [], 'shear_mean' : [], 'tcw_mean' : [], 'tcw_hist' : [], 'shear_hist' : []}
    dic80 = {'model_name' : [], 'shear_mean' : [], 'tcw_mean' : [], 'tcw_hist' : [], 'shear_hist' : []}
    
    dics = [(dic30, '2020-2050'), (dic40, '2030-2060'), (dic60, '2050-2080') , (dic80, '2070-2100')]
    periods = [(2020, 2050, dic30, '2030', 1), (2030, 2060, dic40, '2040', 1), (2050,2080, dic60, '2060', 2), (2070,2100, dic80, '2080', 2)]
    for mn in model_names:

        print('Doing', mn)

        for pp in periods:

            (pp[2])['model_name'].append(mn)

            hmask_dat = xr.open_dataset(glob.glob(tcw_path_fut+'*_'+mn+'_*.nc')[0]).load()

            try:
                hmask_dat =  (hmask_dat['prw'])[(hmask_dat['time.year']>=pp[0]) & (hmask_dat['time.year']<pp[1])].mean('time')
            except:
                ipdb.set_trace()
            hmask = uda.shift_lons_data(hmask_dat)

            for l in lpaths:
                print('Starting', l, mn)
                hp = glob.glob(l[1]+'*_'+mn+'_*.nc')[0]
                fp = glob.glob(l[2]+'*_'+mn+'_*.nc')[0]
                hist = xr.open_dataset(hp)
                fut = xr.open_dataset(fp)

                var = l[3]

                if var not in fut.keys():
                    var = 'unknown'
                try:
                    fut_diff =  (fut[var])[(fut['time.year']>=pp[0]) & (fut['time.year']<pp[1])].mean('time')
                except:
                    ipdb.set_trace()

                diff = np.abs(fut_diff)-np.abs(hist[var])

                shift = uda.shift_lons_data(diff)

                hhist = uda.shift_lons_data(hist[var])

                shift = shift.sel(lon=slice(-25,25), lat=slice(0,30))
                hmask_on_dat = shift.salem.transform(hmask)

                shift.values[(shift.values>500) | (shift.values<-500)]=np.nan
                hhist.values[(hhist.values>500) | (hhist.values<-500)]=np.nan
                
                if var == 'prw':
                    xx = (np.abs(shift)>pp[4])
                else:
                    xx = np.isfinite(shift)
                
                shift = shift.where((hmask_on_dat>=25) & xx)
                hhist = hhist.where(hmask_on_dat>=25) # 27.3 is 0.5th centile threshold for MCS distribution
                
                box = regions[1]
                (pp[2])[l[0]+'_mean'].append(np.float(shift.sel(lon=slice(box[0],box[1]), lat=slice(box[2],box[3])).mean().squeeze().values))   # [-5,15,9,20], Central burkina faso: [-2.5,0,11,13.5]
                (pp[2])[l[0]+'_hist'].append(np.float(hhist.sel(lon=slice(box[0],box[1]), lat=slice(box[2],box[3])).mean().squeeze().values))
                ltag = l[2].split('/')[-2]
                outp = fp.replace(ltag, l[0]+'_diff_'+pp[3])
                #ipdb.set_trace()
                da = xr.DataArray(shift.values[None,...],
                              coords={'model_name': np.array(mn)[None,...], 'lat': shift.lat,
                                      'lon': shift.lon},
                              dims=['model_name', 'lat', 'lon'])

                da.name = var

                da2 = xr.DataArray((hhist).values[None,...],
                  coords={'model_name': np.array(mn)[None,...], 'lat': shift.lat,
                          'lon': shift.lon},
                  dims=['model_name', 'lat', 'lon'])

                da2.name = var+'_hist'

                ds = xr.Dataset()
                ds[var] = da
                ds[var+'_hist'] = da2

    #             shift = shift.assign_attrs({'model_id':mn})
    #             shift = shift.assign_coords({'model_name' : mn})

                if os.path.isfile(outp):
                    os.remove(outp)

                ds.to_netcdf(outp)
                #ipdb.set_trace()
                print('Saving', outp)
    for dd in dics:
        pd.DataFrame.from_dict(dd[0]).to_csv(base+ dd[1]+'_CMIP_'+regions[0]+'.csv')

Doing BNU-ESM
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') BNU-ESM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2030/ua@925-ua@600_20062100_BNU-ESM_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') BNU-ESM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2030/prw_20062100_BNU-ESM_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') BNU-ESM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2040/ua@925-ua@600_20062100_BNU-ESM_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-199

Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2060/ua@925-ua@600_20062100_IPSL-CM5B-LR_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') IPSL-CM5B-LR
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2060/prw_20062100_IPSL-CM5B-LR_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') IPSL-CM5B-LR
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2080/ua@925-ua@600_20062100_IPSL-CM5B-LR_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') IPSL-CM5B-LR
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2080/prw

Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') CSIRO-Mk3-6-0
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2030/ua@925-ua@600_20062300_CSIRO-Mk3-6-0_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') CSIRO-Mk3-6-0
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2030/prw_20062300_CSIRO-Mk3-6-0_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') CSIRO-Mk3-6-0
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2040/ua@925-ua@600_20062300_CSIRO-Mk3-6-0_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP

Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2060/ua@925-ua@600_20062100_CMCC-CMS_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') CMCC-CMS
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2060/prw_20062100_CMCC-CMS_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') CMCC-CMS
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2080/ua@925-ua@600_20062100_CMCC-CMS_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') CMCC-CMS
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2080/prw_20062100_CMCC-CMS_rcp85

/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/home/ck/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: Serializat

...

Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2060/prw_20062300_MPI-ESM-LR_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') MPI-ESM-LR
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2080/ua@925-ua@600_20062300_MPI-ESM-LR_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') MPI-ESM-LR


/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2080/prw_20062300_MPI-ESM-LR_rcp85_r1i1p1_jas_glo.nc
Doing MIROC-ESM-CHEM
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') MIROC-ESM-CHEM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2030/ua@925-ua@600_20062100_MIROC-ESM-CHEM_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') MIROC-ESM-CHEM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2030/prw_20062100_MIROC-ESM-CHEM_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') MIROC-ESM-CHEM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP

Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2040/ua@925-ua@600_20062300_GISS-E2-R_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') GISS-E2-R
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2040/prw_20062300_GISS-E2-R_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') GISS-E2-R
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2060/ua@925-ua@600_20062300_GISS-E2-R_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') GISS-E2-R
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2060/prw_20062300_GISS-E2-

Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2060/prw_20062100_MRI-ESM1_rcp85_r1i1p1_jas_glo.nc
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') MRI-ESM1
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2080/ua@925-ua@600_20062100_MRI-ESM1_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') MRI-ESM1
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2080/prw_20062100_MRI-ESM1_rcp85_r1i1p1_jas_glo.nc


In [12]:
#### calculates full time series CMIP
x=2
ouaga=[-1.52-x,-1.52+x,11.38,13.38]  #12.3714° N, 1.5197° W
niamey = [2.13-x,2.13+x, 12.51,14.51] #13.5116° N, 2.1254° E
timbuktu = [ -3-x,-3+x, 15.77,17.77] #16.7666° N, 3.0026° W

reg_dic = {'timbuktu' : timbuktu, 'ouaga' : ouaga, 'niamey' : niamey} 

for k in reg_dic.keys():
    
    domain = reg_dic[k]

    ts_dic = {'model_name' : [], 'shear_mean' : [], 'tcw_mean' : [], 'tcw_hist' : [], 'shear_hist' : []}

    base = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/'
    shear_path_hist = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/'
    shear_path_fut = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/'
    tcw_path_hist = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/'
    tcw_path_fut = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/'

    lpaths = [('shear', shear_path_hist, shear_path_fut, 'unknown'), ('tcw', tcw_path_hist, tcw_path_fut, 'prw')]

    dics = [(ts_dic, '2020-2100')]

    for mn in model_names:

        print('Doing', mn)


        for l in lpaths:
            var = l[3]
            print('Starting', l, mn)
            hp = glob.glob(l[1]+'*_'+mn+'_*.nc')[0]
            fp = glob.glob(l[2]+'*_'+mn+'_*.nc')[0]
            hist_orig = xr.open_dataset(hp)
            fut_orig = xr.open_dataset(fp)
            
            if var not in fut_orig.keys():
                var = 'unknown'
            
            hist = uda.shift_lons_data(hist_orig[var])
            fut = uda.shift_lons_data(fut_orig[var])
            
            hist = hist.sel(lon=slice(domain[0],domain[1]), lat=slice(domain[2],domain[3])).mean(['lat', 'lon'])     #(-1.52-1,12.38-1),2,2
            fut = fut.sel(lon=slice(domain[0],domain[1]), lat=slice(domain[2],domain[3])).mean(['lat', 'lon'])
            
            #ipdb.set_trace()

            diff = np.abs(fut)-np.abs(hist)

            shift = diff #uda.shift_lons_data(diff)

            hhist = np.abs(hist) #uda.shift_lons_data(hist[var])

            shift.values[(shift.values>500) | (shift.values<-500)]=np.nan
            hhist.values[(hhist.values>500) | (hhist.values<-500)]=np.nan
            
            if var == 'prw':
                xx = (np.abs(shift)>pp[4])
            else:
                xx = np.isfinite(shift)

            shift = shift.where(xx)

            ltag = l[2].split('/')[-2]
            outp = fp.replace(ltag, l[0]+'_diff_2020-2100_'+k)
           # ipdb.set_trace()
            da = xr.DataArray(shift.values[None,...],
                          coords={'model_name': np.array(mn)[None,...], 'time': shift.time},
                          dims=['model_name', 'time'])

            da.name = var

            da2 = xr.DataArray((hhist).values[None,...],
              coords={'model_name': np.array(mn)[None,...]},
                          dims=['model_name'])

            da2.name = var+'_hist'

            ds = xr.Dataset()
            ds[var] = da
            ds[var+'_hist'] = da2

            if os.path.isfile(outp):
                os.remove(outp)

            ds.to_netcdf(outp)
            print('Saving', outp)

Doing CESM1-BGC
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') CESM1-BGC
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_timbuktu/ua@925-ua@600_20062100_CESM1-BGC_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') CESM1-BGC
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2020-2100_timbuktu/prw_20062100_CESM1-BGC_rcp85_r1i1p1_jas_glo.nc
Doing inmcm4
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') inmcm4
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_timbuktu/ua@925-ua@600_20062100_inmcm4_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '

Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_timbuktu/ua@925-ua@600_20062100_CanESM2_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') CanESM2
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2020-2100_timbuktu/prw_20062100_CanESM2_rcp85_r1i1p1_jas_glo.nc
Doing NorESM1-M
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') NorESM1-M
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_timbuktu/ua@925-ua@600_20062100_NorESM1-M_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') NorESM1-M
Saving /media/ck/Elements/Africa/WestAfri

/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2020-2100_timbuktu/prw_20062300_GISS-E2-H_rcp85_r1i1p1_jas_glo.nc
Doing CNRM-CM5
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') CNRM-CM5
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_timbuktu/ua@925-ua@600_20062300_CNRM-CM5_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') CNRM-CM5


/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


...

Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2020-2100_niamey/prw_20062300_CNRM-CM5_rcp85_r1i1p1_jas_glo.nc
Doing CMCC-CESM
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') CMCC-CESM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_niamey/ua@925-ua@600_20062100_CMCC-CESM_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') CMCC-CESM
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2020-2100_niamey/prw_20062100_CMCC-CESM_rcp85_r1i1p1_jas_glo.nc
Doing MPI-ESM-LR
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') MPI-ESM-LR


/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_niamey/ua@925-ua@600_20062300_MPI-ESM-LR_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') MPI-ESM-LR


/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/ck/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2020-2100_niamey/prw_20062300_MPI-ESM-LR_rcp85_r1i1p1_jas_glo.nc
Doing MPI-ESM-MR
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') MPI-ESM-MR
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_diff_2020-2100_niamey/ua@925-ua@600_20062100_MPI-ESM-MR_rcp85_r1i1p1_jas_glo.nc
Starting ('tcw', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_2006-2100/', 'prw') MPI-ESM-MR
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/tcw_diff_2020-2100_niamey/prw_20062100_MPI-ESM-MR_rcp85_r1i1p1_jas_glo.nc
Doing GISS-E2-R-CC
Starting ('shear', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_1950-1990/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/shear_2006-2100/', 'unknown') GISS-E2-R-CC
Saving /media/c

In [11]:
base = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/'
# pr_hist = '/media/ck/Elements/Africa/WestAfrica/CMIP5/BC_CMIP5_0.5x0.5_rain/'
# pr_fut = '/media/ck/Elements/Africa/WestAfrica/CMIP5/BC_CMIP5_0.5x0.5_rain/'

pr_hist = '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/'
pr_fut = '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/'

lpaths = [('pr', pr_hist, pr_fut, 'pr')]

sahel = [-10,15,9,19]
x=2
y=1.1
ouaga=[-1.52-x,-1.52+x,12.38-y,12.38+y]  #12.3714° N, 1.5197° W
niamey = [2.13-x,2.13+x, 13.51-y,13.51+y] #13.5116° N, 2.1254° E
timbuktu = [ -3-x,-3+x, 16.77-y,16.77+y] #16.7666° N, 3.0026° W
bamako = [-8-x, -8+x, 12.6-y, 12.6+y]
dakar = [-16.9-x, -16.9+x, 14.8-y, 14.8+y]

dnames = [('dakar', dakar)] #('ouaga',  ouaga), ('niamey', niamey), ('timbuktu',  timbuktu) ]   #('sahel', sahel), 
for regions in dnames:
    dic30 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    dic40 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    dic60 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    dic80 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    
    dics = [(dic80, '2070-2100')]  #(dic30, '2020-2050'), (dic40, '2030-2060'), (dic60, '2050-2080') , 
    periods = [(2070,2100, dic80, '2080', 2)]        #(2020, 2050, dic30, '2030', 1), (2030, 2060, dic40, '2040', 1), (2050,2080, dic60, '2060', 2), 
    for mn in model_names:

        print('Doing', mn)

        for pp in periods:

            

            for l in lpaths:
                print('Starting', l, mn)
                try:
                    hp = glob.glob(l[1]+'*_'+mn+'*_historical_*.nc')[0]
                except:
                    print('Model not found, continue')
                    continue
                    
                (pp[2])['model_name'].append(mn)
                fp = glob.glob(l[2]+'*_'+mn+'_*_rcp85_*.nc')[0]
                hist = xr.open_dataset(hp)
                fut = xr.open_dataset(fp)

                var = l[3]

                if var not in fut.keys():
                    var = 'unknown'
                    
                try:
                    maski = (hist['time.year']>=1950) & (hist['time.year']<=1999) & (hist['time.month']>=7) & (hist['time.month']<=9)
                    histinter =  ((hist[var])).sel(lon=slice(-25,25), lat=slice(0,30)).isel(time=maski)*3600
                   
                    histinter = histinter.where(histinter>0.1)  # remove low rainfall days
                    valmask = np.sum(np.isfinite(histinter.values), axis=0) / histinter.shape[0] 
                
                    hist_diff = histinter.quantile(0.95, dim='time', skipna=True)
                    hist_diff.values[valmask<0.01] = np.nan # at lest 2% 
                except:
                    ipdb.set_trace()
                    
                try:
                    maski = (fut['time.year']>=pp[0]) & (fut['time.year']<pp[1])
                    futinter =  ((fut[var])).sel(lon=slice(-25,25), lat=slice(0,30)).isel(time=maski)*3600
                    futinter = futinter.where(futinter>0.1)
                    valmask = np.sum(np.isfinite(futinter.values), axis=0) / futinter.shape[0] 
                
                    fut_diff = futinter.quantile(0.95, dim='time', skipna=True)
                    fut_diff.values[valmask<0.01] = np.nan
                except:
                    ipdb.set_trace()
                #ipdb.set_trace()
                diff = np.abs(fut_diff)-np.abs(hist_diff)

                shift = diff# uda.shift_lons_data(diff)

                hhist = hist_diff #uda.shift_lons_data(hist[var])

                shift = shift
                hhist = hhist

                shift.values[(shift.values>500) | (shift.values<-500)]=np.nan
                hhist.values[(hhist.values>500) | (hhist.values<-500)]=np.nan
                
                box = regions[1]
                (pp[2])[l[0]+'_mean'].append(np.float(shift.sel(lon=slice(box[0],box[1]), lat=slice(box[2],box[3])).mean().squeeze().values))   # [-5,15,9,20], Central burkina faso: [-2.5,0,11,13.5]
                (pp[2])[l[0]+'_hist'].append(np.float(hhist.sel(lon=slice(box[0],box[1]), lat=slice(box[2],box[3])).mean().squeeze().values))
                ltag = l[2].split('/')[-2]
                outp = fp.replace(ltag, l[0]+'_diff_'+pp[3])
                outp = outp.replace('/CMIP5/', '/CMIP5/endgame/')
                #ipdb.set_trace()
                da = xr.DataArray(shift.values[None,...],
                              coords={'model_name': np.array(mn)[None,...], 'lat': shift.lat,
                                      'lon': shift.lon},
                              dims=['model_name', 'lat', 'lon'])

                da.name = var

                da2 = xr.DataArray((hhist).values[None,...],
                  coords={'model_name': np.array(mn)[None,...], 'lat': shift.lat,
                          'lon': shift.lon},
                  dims=['model_name', 'lat', 'lon'])

                da2.name = var+'_hist'

                ds = xr.Dataset()
                ds[var] = da
                ds[var+'_hist'] = da2

    #             shift = shift.assign_attrs({'model_id':mn})
    #             shift = shift.assign_coords({'model_name' : mn})
                print('OUTP', outp)
                if os.path.isfile(outp):
                    os.remove(outp)

                ds.to_netcdf(outp)
                #ipdb.set_trace()
                print('Saving', outp)
   # ipdb.set_trace()
    for dd in dics:
        pd.DataFrame.from_dict(dd[0]).to_csv(base+ dd[1]+'_RAW_CMIP_'+regions[0]+'.csv')
        print('Written DIR!')

Doing BNU-ESM
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') BNU-ESM
OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_BNU-ESM_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_BNU-ESM_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing FIO-ESM
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') FIO-ESM
Model not found, continue
Doing CanESM2
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') CanESM2


/home/ck/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_CanESM2_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_CanESM2_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing IPSL-CM5B-LR
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') IPSL-CM5B-LR
OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_IPSL-CM5B-LR_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_IPSL-CM5B-LR_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing CESM1-BGC
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') CESM1-BGC
Model not found, continue
Doing NorESM1-M
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain

/home/ck/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_NorESM1-M_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_NorESM1-M_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing HadGEM2-ES
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') HadGEM2-ES


...

OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MPI-ESM-LR_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MPI-ESM-LR_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing MIROC-ESM-CHEM
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') MIROC-ESM-CHEM
OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MIROC-ESM-CHEM_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MIROC-ESM-CHEM_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing bcc-csm1-1-m
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') bcc-csm1-1-m
OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_bcc-csm1-1-m_west-africa

/home/ck/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MPI-ESM-MR_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MPI-ESM-MR_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing GISS-E2-R
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') GISS-E2-R
Model not found, continue
Doing GFDL-CM3
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') GFDL-CM3


/home/ck/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_GFDL-CM3_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_GFDL-CM3_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Doing GISS-E2-H-CC
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') GISS-E2-H-CC
Model not found, continue
Doing MRI-ESM1
Starting ('pr', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', '/media/ck/Elements/Africa/WestAfrica/CMIP5/CMIP5_0.5x0.5_rain/', 'pr') MRI-ESM1
OUTP /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MRI-ESM1_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Saving /media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/pr_diff_2080/pr_day_MRI-ESM1_west-africa_0.5x0.5_rcp85_r1i1p1_full.nc
Written DIR!


In [ ]:
base = '/media/ck/Elements/Africa/WestAfrica/CMIP5/endgame/'
pr_hist = '/media/ck/Elements/Africa/WestAfrica/CMIP5/BC_CMIP5_0.5x0.5_rain/'
pr_fut = '/media/ck/Elements/Africa/WestAfrica/CMIP5/BC_CMIP5_0.5x0.5_rain/'

lpaths = [('pr', pr_hist, pr_fut, 'pr')]

sahel = [-10,15,9,19]
x=2
ouaga=[-1.52-x,-1.52+x,11.38,13.38]  #12.3714° N, 1.5197° W
niamey = [2.13-x,2.13+x, 12.51,14.51] #13.5116° N, 2.1254° E
timbuktu = [ -3-x,-3+x, 15.77,17.77] #16.7666° N, 3.0026° W

dnames = [('sahel', sahel), ('ouaga',  ouaga), ('niamey', niamey), ('timbuktu',  timbuktu) ]
for regions in dnames:
    dic30 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    dic40 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    dic60 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    dic80 = {'model_name' : [], 'pr_mean' : [], 'pr_hist' : []}
    
    dics = [(dic30, '2020-2050'), (dic40, '2030-2060'), (dic60, '2050-2080') , (dic80, '2070-2100')]
    periods = [(2020, 2050, dic30, '2030', 1), (2030, 2060, dic40, '2040', 1), (2050,2080, dic60, '2060', 2), (2070,2100, dic80, '2080', 2)]
    for mn in model_names:

        print('Doing', mn)

        for pp in periods:

            

            for l in lpaths:
                print('Starting', l, mn)
                try:
                    hp = glob.glob(l[1]+'*_'+mn+'*_historical_*.nc')[0]
                except:
                    print('Model not found, continue')
                    continue
                    
                (pp[2])['model_name'].append(mn)
                fp = glob.glob(l[2]+'*_'+mn+'*_rcp85_*.nc')[0]
                hist = xr.open_dataset(hp)
                fut = xr.open_dataset(fp)

                var = l[3]

                if var not in fut.keys():
                    var = 'unknown'
                    
                try:
                    maski = (hist['time.year']>=1950) & (hist['time.year']<=1999)
                    histinter =  ((hist[var])).sel(lon=slice(-25,25), lat=slice(0,30)).isel(time=maski)*3600
                    histinter = histinter.where(histinter>0.1)
                    valmask = np.sum(np.isfinite(histinter.values), axis=0) / histinter.shape[0] 
                
                    hist_diff = histinter.quantile(0.95, dim='time', skipna=True)
                    hist_diff.values[valmask<0.02] = np.nan
                except:
                    ipdb.set_trace()
                    
                try:
                    maski = (fut['time.year']>=pp[0]) & (fut['time.year']<pp[1])
                    futinter =  ((fut[var])).sel(lon=slice(-25,25), lat=slice(0,30)).isel(time=maski)*3600
                    futinter = futinter.where(futinter>0.1)
                    valmask = np.sum(np.isfinite(futinter.values), axis=0) / futinter.shape[0] 
                
                    fut_diff = futinter.quantile(0.95, dim='time', skipna=True)
                    fut_diff.values[valmask<0.02] = np.nan
                except:
                    ipdb.set_trace()

                diff = np.abs(fut_diff)-np.abs(hist_diff)

                shift = diff# uda.shift_lons_data(diff)

                hhist = hist_diff #uda.shift_lons_data(hist[var])

                shift = shift
                hhist = hhist

                shift.values[(shift.values>500) | (shift.values<-500)]=np.nan
                hhist.values[(hhist.values>500) | (hhist.values<-500)]=np.nan
                
                box = regions[1]
                (pp[2])[l[0]+'_mean'].append(np.float(shift.sel(lon=slice(box[0],box[1]), lat=slice(box[2],box[3])).mean().squeeze().values))   # [-5,15,9,20], Central burkina faso: [-2.5,0,11,13.5]
                (pp[2])[l[0]+'_hist'].append(np.float(hhist.sel(lon=slice(box[0],box[1]), lat=slice(box[2],box[3])).mean().squeeze().values))
                ltag = l[2].split('/')[-2]
                outp = fp.replace(ltag, l[0]+'_diff_'+pp[3])
                outp = outp.replace('/CMIP5/', '/CMIP5/endgame/')
                #ipdb.set_trace()
                da = xr.DataArray(shift.values[None,...],
                              coords={'model_name': np.array(mn)[None,...], 'lat': shift.lat,
                                      'lon': shift.lon},
                              dims=['model_name', 'lat', 'lon'])

                da.name = var

                da2 = xr.DataArray((hhist).values[None,...],
                  coords={'model_name': np.array(mn)[None,...], 'lat': shift.lat,
                          'lon': shift.lon},
                  dims=['model_name', 'lat', 'lon'])

                da2.name = var+'_hist'

                ds = xr.Dataset()
                ds[var] = da
                ds[var+'_hist'] = da2

    #             shift = shift.assign_attrs({'model_id':mn})
    #             shift = shift.assign_coords({'model_name' : mn})
                print('OUTP', outp)
                if os.path.isfile(outp):
                    os.remove(outp)

                ds.to_netcdf(outp)
                #ipdb.set_trace()
                print('Saving', outp)

    for dd in dics:
        pd.DataFrame.from_dict(dd[0]).to_csv(base+ dd[1]+'_BC_CMIP_'+regions[0]+'.csv')